# Advanced Indexing

## 멀티 인덱스 (multi-index) 
인덱스가 여러 개인 `DataFrame`

In [1]:
from pandas import DataFrame

data = [
    ["2019", "A", 300, 5],
    ["2019", "B", 200, 4],
    ["2019", "C", 100, 8],
    ["2020", "A", 400, 8],
    ["2020", "B", 230, 3],    
]
columns =['date', 'item', 'price', 'volume']
df = DataFrame(data=data, columns=columns)
df

,date,item,price,volume
0,2019,A,300,5
1,2019,B,200,4
2,2019,C,100,8
3,2020,A,400,8
4,2020,B,230,3


In [2]:
df['item'].nunique()

3

### 멀티 인덱스 생성

이차원 인덱스를 사용할 수 있다면 응용해서 다차원으로 확장 적용할 수 있습니다.  
`date`와 `item`을 이차원 인덱스로 지정해 봅시다.

In [ ]:
df.set_index('date')

,item,price,volume
date,,,
2019,A,300,5
2019,B,200,4
2019,C,100,8
2020,A,400,8
2020,B,230,3


In [4]:
t = df.set_index(   ['date', 'item']    )

index의 `get_level_values( 숫자 )`로 값을 확인할 수 있음

<img src="https://i.ibb.co/VJmLnKT/pandas-2-0.png" width="230" style="float:left" />

왼쪽위에 표시되는 이름은 `index` 객체의 `names` 속성에 저장된 값

In [ ]:
df.index.names

FrozenList(['date', 'item'])

### 칼럼 인덱싱과 슬라이싱

칼럼을 인덱싱하면 기존 데이타프레임과 동일한 인덱스와 값이 출력됩니다. 이 때 출력되는 결과는 멀티 인덱스를 같는 `Series` 객체입니다. 

In [ ]:
t[ 'price' ].to_frame()

price
date item       
2019 A       300
     B       200
     C       100
2020 A       400
     B       230

<img src="https://i.ibb.co/6m3kNJL/pandas-2-1.png" width="200" style="float:left" />

In [ ]:
t['volume']

date  item
2019  A       5
      B       4
      C       8
2020  A       8
      B       3
Name: volume, dtype: int64

<img src="https://i.ibb.co/KWtmGXy/pandas-2-2.png" width="200" style="float:left" />

`인덱스 값이 동일할때 생략(자동?)인가요?`

내부적으로는 모두 저장돼 있습니다.


In [ ]:
t.index

MultiIndex([('2019', 'A'),
            ('2019', 'B'),
            ('2019', 'C'),
            ('2020', 'A'),
            ('2020', 'B')],
           names=['date', 'item'])

`2019랑 2020이 뒤죽박죽이면 정리되서 저장되나요?`


In [ ]:
b = df.iloc[ [0, 4, 2, 3, 1] ].set_index( ['date', 'item'] )
b.sort_index(ascending=False)

price  volume
date item               
2020 B       230       3
     A       400       8
2019 C       100       8
     B       200       4
     A       300       5

In [ ]:
b.sort_values('price', ascending=False)

price  volume
date item               
2020 A       400       8
2019 A       300       5
2020 B       230       3
2019 B       200       4
     C       100       8

칼럼의 슬라이싱은 출력되는 결과가 멀티 인덱스를 갖는 것이외에 싱글 인덱스의 칼럼 슬라이싱과 다르지 않습니다.  


### 로우 인덱싱과 슬라이싱

`loc`를 사용해서 하나의 인덱스로 인덱싱할 수 있습니다.  
인덱스는 level 0부터 사용합니다. 

In [ ]:
t.loc[ "2019" ]

,price,volume
item,,
A,300,5
B,200,4
C,100,8


<img src="https://i.ibb.co/V3F1swK/pandas-2-3.png" width="200" style="float:left" />

`loc`로 인덱싱을 한 뒤에 반환되는 결과는 1차원 인덱스를 갖는 데이터프레임 입니다.  
재차 인덱싱을 적용해서 원하는 결과를 가져올 수 있습니다. 

In [ ]:
t.loc["2019"].loc["A"]

price     300
volume      5
Name: A, dtype: int64

In [ ]:
t.loc['2019', 'A']

price     300
volume      5
Name: (2019, A), dtype: int64

In [ ]:
t.loc[ ('2019', 'A') ]

price     300
volume      5
Name: (2019, A), dtype: int64

<img src="https://i.ibb.co/DMqtBnw/pandas-2-4.png" width="200" style="float:left" />

In [ ]:
t.loc[ ('2019', 'A') ]

price     300
volume      5
Name: (2019, A), dtype: int64

In [ ]:
%timeit  t.loc["2019"].loc["A"].iloc[0]

The slowest run took 5.36 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 5: 365 µs per loop


In [ ]:
%timeit t.iloc[0, 0]

The slowest run took 7.66 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 5: 28.4 µs per loop


In [ ]:
%timeit t.loc[('2019', 'A'),'price']

The slowest run took 5.97 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 5: 60 µs per loop


In [ ]:
# 왜안되요?
t.loc['2019'].loc["A"].to_frame()

,A
price,300
volume,5


한 번에 이차원 인덱스의 인덱싱은 튜플로 이차원 인덱스를 지정합니다.

In [6]:
t.loc[('2019', 'A'),'price']

300

###### 연속적인 행 선택하기  
`loc`의 슬라이싱은 지정해준 인덱스의 시작과 끝을 포함한 사이에 있는 모든 데이터를 가져옵니다. 
이차원 인덱스 이므로 튜플로 인덱스를 지정합니다. 

In [ ]:
t.loc[:, :]

price  volume
date item               
2019 A       300       5
     B       200       4
     C       100       8
2020 A       400       8
     B       230       3

###### 불연속적인 행을 가져오기  
`df.loc[ [] , []]`

가져오려는 데이터의 인덱스를 튜플 (혹은 리스트) 로 지정합니다. 

In [ ]:
t.loc[ [("2019", "A"), ("2020", "A")], :]

,,price,volume
date,item,,
2019,A,300,5
2020,A,400,8


###### 특정 레벨의 모든 데이터 선택하기
`slice(None)`을 사용한 모든 `B`를 선택

In [ ]:
# t.loc[  (level0, level1), ~ ]

price  volume
date item               
2019 A       300       5
     B       200       4
     C       100       8
2020 A       400       8
     B       230       3

In [ ]:
t.loc[   (  slice(None) ,  "B" )  , :  ]

,,price,volume
date,item,,
2019,B,200,4
2020,B,230,3


`level 0을 : 요렇게 슬라이싱하면 안되나요?`


In [ ]:
t.loc['2019'].loc["B"]

price     200
volume      4
Name: B, dtype: int64

In [ ]:
t.loc['2020'].loc["B"]

price     230
volume      3
Name: B, dtype: int64

In [ ]:
df.loc['2019', (slice(None)), : ]

price  volume
date item               
2019 A       300       5
     B       200       4
     C       100       8

 왜 셋다 될까요?

```python
t.loc['2019', slice(None), : ]
t.loc['2019', (slice(None)), : ]
t.loc[('2019', slice(None)), : ]
```

In [ ]:
t.loc['2019', ((slice(None))), : ]

price  volume
date item               
2019 A       300       5
     B       200       4
     C       100       8

In [ ]:
3, 2

(3, 2)

In [ ]:
# 얘는 안되더라고요
t.loc[("2019", "A"), "price"] 


300

### 로우 데이터 추가
이차원 인덱스의 위치 정보를 차례로 지정해야 합니다.  
칼럼 정보(`:`)까지 입력하는것에 주의해야 합니다. 

In [ ]:
df

<img src="https://i.ibb.co/1nBH2Pd/pandas-2-6.png" width="200" style="float:left" />

### 데이터 정렬
멀티 인덱스에서 데이터가 정렬돼 있어야 효과적입니다. 

In [ ]:
import numpy as np

data = [
    ["2019", "A", 300, 5],
    ["2019", "B", 200, 4],
    ["2019", "C", 100, 8],
    ["2020", "A", 400, 8],
    ["2020", "B", 230, 3],    
]


np.random.shuffle(data)

columns = ['date', 'item', 'price', 'volume']
df = DataFrame(data=data, columns=columns)
df = df.set_index(['date', 'item'])
df

price  volume
date item               
2020 A       400       8
2019 C       100       8
2020 B       230       3
2019 B       200       4
     A       300       5

In [ ]:
df_sorted = df.sort_index()

In [ ]:
%timeit df.loc['2019']

216 µs ± 3.69 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
%timeit df_sorted.loc['2019']

113 µs ± 1.16 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## Multi Columns

인덱스와 유사하게 2 level로 구성된 멀티 칼럼을 실습해 봅시다.

In [ ]:
data = [
    ["삼성전자", 300, 2000, 700, 300, 400],
    ["Naver", 200, 4000, 500, 300, 200],
    ["LG전자", 100, 1200, 300, 200, 100],        
]

columns = ['종목', '종가', '거래량', '자산', '자본', '부채']
df = DataFrame(data=data, columns=columns)
df = df.set_index('종목')
df

,종가,거래량,자산,자본,부채
종목,,,,,
삼성전자,300,2000,700,300,400
Naver,200,4000,500,300,200
LG전자,100,1200,300,200,100


In [ ]:
df.columns = [ ['시장정보', '시장정보', '재무상태', '재무상태', '재무상태'], 
               ['종가', '거래량', '자산', '자본', '부채'] ]

In [ ]:
df

시장정보       재무상태          
        종가   거래량   자산   자본   부채
종목                             
삼성전자   300  2000  700  300  400
Naver  200  4000  500  300  200
LG전자   100  1200  300  200  100

In [ ]:
%timeit df.iloc[  0,    1 ]

The slowest run took 6.83 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 5: 45.3 µs per loop


In [ ]:
%timeit df.loc[  "삼성전자"  ,  ('시장정보' , '거래량'  )  ]

The slowest run took 6.31 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 5: 126 µs per loop


In [ ]:
df['시장정보'].loc['삼성전자', '거래량']

2000

In [ ]:
df['시장정보'].loc['삼성전자', '거래량'] = 1000
df

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


시장정보       재무상태          
        종가   거래량   자산   자본   부채
종목                             
삼성전자   300  1000  700  300  400
Naver  200  4000  500  300  200
LG전자   100  1200  300  200  100

In [ ]:
df.시장정보.loc['삼성전자','거래량']
df["시장정보"].loc['삼성전자','거래량']

1000

### 인덱싱과 슬라이싱

멀티 인덱스와 유사합니다. 사용해 봅시다. 

###### 연속적인 칼럼 슬라이싱

###### 불연속적인 칼럼 슬라이싱

###### 특정 레벨의 모든 칼럼 슬라이싱

## Transpose